In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#importing various parts of the data below. print the columns as well.

In [12]:
ordered_items = pd.read_csv("~/Desktop/archive (5)/olist_order_items_dataset.csv")
ordered_items.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [13]:
payments = pd.read_csv("~/Desktop/archive (5)/olist_order_payments_dataset.csv")
payments.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')

In [14]:
review_scores = pd.read_csv("~/Desktop/archive (5)/olist_order_reviews_dataset.csv")
review_scores.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [15]:
delivery_info = pd.read_csv("~/Desktop/archive (5)/olist_orders_dataset.csv")
delivery_info.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [16]:
product_info = pd.read_csv("~/Desktop/archive (5)/olist_products_dataset.csv")
product_info.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [17]:
product_category_translation = pd.read_csv("~/Desktop/archive (5)/product_category_name_translation.csv")
product_category_translation.columns

Index(['product_category_name', 'product_category_name_english'], dtype='object')

In [82]:
#importing the translating package so I can translate the portugese reviews
import translators as ts


In [80]:
#take a sample of the reviews
reviews =review_scores['review_comment_message'].dropna()[:10]

In [81]:
#iterate through the 10 samples to see if the translation isn't through off by non utf 8 charachters.
for review in reviews:
    line = ts.translate_text(review)
    print(line)

I received it well before the stipulated deadline.
Congratulations lannister stores loved buying over the Internet safe and practical Happy birthday to all Happy Easter
efficient appliance. On the website the brand of the device is printed as 3Disinfector and when it arrives it is under another name... update with the correct brand since it is the same handset
But a little bit, crashing... for the value it's Good.
Reliable seller, ok product and delivery before the deadline.
I WOULD LIKE TO KNOW WHAT HAPPENED, I HAVE ALWAYS RECEIVED IT AND THIS PURCHASE HAS NOW DISAPPOINTED ME
Lousy
Shop Note 10
Thank you for your attention,
The purchase was made easily.
The delivery was made well ahead of schedule.
The product has already started to be used and to date,
No problem.
